In [3]:
import pandas as pd
import os

In [44]:
import json

In [69]:
from IPython.display import Markdown

In [143]:
import numpy as np

In [2]:
folder = "/mnt/disk3/maalfrid_government/src"

In [9]:
files = [f"{os.path.join(folder,x)}" for x in os.listdir(folder) if x.startswith('regjeringen_doc')]

In [10]:
files

['/mnt/disk3/maalfrid_government/src/regjeringen_docs_2000.01.01_2009.12.31_archived.jsonl',
 '/mnt/disk3/maalfrid_government/src/regjeringen_docs_2000.01.01_2009.12.31_unarchived.jsonl',
 '/mnt/disk3/maalfrid_government/src/regjeringen_docs_2010_2024_archived.jsonl',
 '/mnt/disk3/maalfrid_government/src/regjeringen_docs_1814.01.01_1999.12.31_unarchived.jsonl',
 '/mnt/disk3/maalfrid_government/src/regjeringen_docs_2010_2024_unarchived.jsonl',
 '/mnt/disk3/maalfrid_government/src/regjeringen_docs_1814.01.01_1999.12.31_archived.jsonl']

In [161]:
df_regjering = pd.DataFrame()
for file in files:
    chunks = pd.read_json(file, lines = True, chunksize=10000)
    for chunk in chunks:
        df_regjering = pd.concat([df_regjering, chunk[chunk.pageType.str.contains("Høring|Høyring")]])

In [165]:
df_regjering.sample(3)

,documentId,uniqueId,language,masterLanguage,relatedLanguages,title,date,documentUrl,pageType,period,otherFormatType,publisher,additionalPublishers,hasContent,doc_mode,fulltext,filepath,additional_docs,owner,additionalOwners
8194,2005783,HDK201420140068___DDDEPIS,nb,nb,[],Høring – endringer i bostøtteloven,2014-10-08 00:00:00,https://www.regjeringen.no/no/dokumenter/Horin...,Høring,2014,None,NaN,NaN,True,html,[Kommunal- og moderniseringsdepartementet send...,mandat/html/83e718b48076b8ee3556c1cb82b733837d...,"{""main"": {""/globalassets/upload/kmd/boby/bosto...","{'epiId': '504', 'depCode': 'kdd', 'depName': ...",[]
5446,271028,HDK200620060027JD_DDDEPIS,nb,nb,[],Høring - hurtigere behandling av straffesaker,2006-09-28 00:00:00,https://www.regjeringen.no/no/dokumenter/horin...,Høring,2006,None,None,None,True,html,[Vedlagt følger høringsnotat\nseptember 2006 o...,mandat/html/70bb203706549dcb288ec990f0a005e4ff...,"{""main"": {""/globalassets/upload/jd/vedlegg/hor...","{'epiId': '463', 'depCode': 'jd', 'depName': '...",[]
10994,2426000,None,nb,nb,[],Høring av rapport fra markedsbalanseringsutval...,2015-07-01 00:00:00,https://www.regjeringen.no/no/dokumenter/horin...,Høring,2015,None,None,None,True,html,"[Markedsbalanseringsutvalget, ledet av profess...",mandat/html/78d1723fe380847d952ccfdbf4530841b7...,"{""main"": {}, ""attachments"": {}}","{'epiId': '627', 'depCode': 'lmd', 'depName': ...",[]


In [66]:
fs = []
for x in json.loads(df.additional_docs.values[4000])['main'].values():
    fs.append(os.path.join(folder, "mandat", "pdf_markdown", os.path.basename(x).replace('pdf', 'md')) )

In [163]:
publishers = df_regjering[df_regjering.pageType.str.contains("Høring|Høyring")][['documentId','publisher']]

In [164]:
owners = df_regjering[df_regjering.pageType.str.contains("Høring|Høyring")][['documentId','owner']]

In [166]:
publishers.publisher

22588    {'epiId': '279851', 'depCode': 'aid', 'depName...
23701    {'epiId': '279851', 'depCode': 'aid', 'depName...
28854    {'epiId': '279869', 'depCode': 'fin', 'depName...
30432    {'epiId': '232462', 'depCode': 'ud', 'depName'...
30863    {'epiId': '279851', 'depCode': 'aid', 'depName...
                               ...                        
8292     {'epiId': '231602', 'depCode': 'kd', 'depName'...
8294     {'epiId': '231602', 'depCode': 'kd', 'depName'...
8749     {'epiId': '231602', 'depCode': 'kd', 'depName'...
10156    {'epiId': '231602', 'depCode': 'kd', 'depName'...
10360    {'epiId': '231602', 'depCode': 'kd', 'depName'...
Name: publisher, Length: 6516, dtype: object

In [168]:
owners.owner.dropna()

10       {'epiId': '668', 'depCode': 'kld', 'depName': ...
11       {'epiId': '463', 'depCode': 'jd', 'depName': '...
15       {'epiId': '421', 'depCode': 'hod', 'depName': ...
18       {'epiId': '463', 'depCode': 'jd', 'depName': '...
21       {'epiId': '216', 'depCode': 'fin', 'depName': ...
                               ...                        
40404    {'epiId': '216', 'depCode': 'fin', 'depName': ...
40416    {'epiId': '165', 'depCode': 'aid', 'depName': ...
40534    {'epiId': '627', 'depCode': 'lmd', 'depName': ...
40535    {'epiId': '165', 'depCode': 'aid', 'depName': ...
40552    {'epiId': '463', 'depCode': 'jd', 'depName': '...
Name: owner, Length: 6234, dtype: object

In [169]:
import pandas as pd

# Use list comprehension to filter out entries that are not dictionaries
filtered_pub = [{'documentId':y.id, **x} for y in publishers.publisher if isinstance(x, dict)]
filtered_own = [{'documentId':y.id, **x} for y in owners.owner if isinstance(x, dict)]
# Create DataFrame from the filtered list of dictionaries
pub_df = pd.DataFrame(filtered_pub)
own_df = pd.DataFrame(filtered_own)

In [184]:
publishers_df = publishers['publisher'].apply(pd.Series)
owners_df = owners['owner'].apply(pd.Series)

# Ensure both DataFrames have 'documentId' to join on
publishers_df['documentId'] = publishers['documentId']
owners_df["documentId"] = owners['documentId']

# Join the DataFrames with høring
høring_expanded = pd.merge(høring, publishers_df, on='documentId', how='left', suffixes=('', '_pub'))
expand = pd.merge(høring_expanded, owners_df, on='documentId', how='left', suffixes=('', '_own'))
expand['depName'] = expand['depName'].fillna(expand['depName_own'])
expand['depCode'] = expand['depCode'].fillna(expand['depCode_own'])

In [185]:
expanded = expand[['documentId', 'depCode', 'depName']]

In [202]:
set(expanded.depName)

{'Arbeids- og administrasjonsdepartementet',
 'Arbeids- og inkluderingsdepartementet',
 'Arbeids- og sosialdepartementet',
 'Barne- og familiedepartementet',
 'Digitaliserings- og forvaltningsdepartementet',
 'Energidepartementet',
 'Finansdepartementet',
 'Fiskeridepartementet',
 'Forsvarsdepartementet',
 'Helse- og omsorgsdepartementet',
 'Helsedepartementet',
 'Justis- og beredskapsdepartementet',
 'Justis- og politidepartementet',
 'Kirke-, utdannings- og forskningsdepartementet',
 'Klima- og miljødepartementet',
 'Kommunal- og distriktsdepartementet',
 'Kommunal- og regionaldepartementet',
 'Kultur- og kirkedepartementet',
 'Kultur- og likestillingsdepartementet',
 'Kulturdepartementet',
 'Kunnskapsdepartementet',
 'Landbruks- og matdepartementet',
 'Landbruksdepartementet',
 'Miljøverndepartementet',
 'Nærings- og fiskeridepartementet',
 'Nærings- og handelsdepartementet',
 'Olje- og energidepartementet',
 'Samferdselsdepartementet',
 'Sosial- og helsedepartementet',
 'Sosialdepa

In [203]:
expanded[expanded.depName.str.contains('Kunnskap|Kirke')]

,documentId,depCode,depName
140,421559,kuf,"Kirke-, utdannings- og forskningsdepartementet"
178,232755,kuf,"Kirke-, utdannings- og forskningsdepartementet"
224,232736,kuf,"Kirke-, utdannings- og forskningsdepartementet"
256,588785,kd,Kunnskapsdepartementet
268,586826,kd,Kunnskapsdepartementet
...,...,...,...
6423,2965159,kd,Kunnskapsdepartementet
6453,2961535,kd,Kunnskapsdepartementet
6465,2959688,kd,Kunnskapsdepartementet
6474,2958960,kd,Kunnskapsdepartementet


In [206]:
df_regjering.merge(expanded, left_on='documentId', right_on='documentId').to_csv('høringer.csv')

In [210]:
df_regjering.merge(expanded, left_on='documentId', right_on='documentId').to_csv('høringer.csv')

In [94]:
høringer = høring.owner.dropna()

In [100]:
høringer

10       {'epiId': '668', 'depCode': 'kld', 'depName': ...
11       {'epiId': '463', 'depCode': 'jd', 'depName': '...
15       {'epiId': '421', 'depCode': 'hod', 'depName': ...
18       {'epiId': '463', 'depCode': 'jd', 'depName': '...
21       {'epiId': '216', 'depCode': 'fin', 'depName': ...
                               ...                        
40404    {'epiId': '216', 'depCode': 'fin', 'depName': ...
40416    {'epiId': '165', 'depCode': 'aid', 'depName': ...
40534    {'epiId': '627', 'depCode': 'lmd', 'depName': ...
40535    {'epiId': '165', 'depCode': 'aid', 'depName': ...
40552    {'epiId': '463', 'depCode': 'jd', 'depName': '...
Name: owner, Length: 6234, dtype: object